In [17]:
import os
import pandas as pd
import numpy as np
from tqdm import trange, tqdm

from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

from pandas import read_csv
from scipy import stats

import torch
import torch.nn as nn

import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import RandomSampler

In [5]:

# Prepare the data for the DeepAR model
def prep_data(data, covariates, data_start, train = True):
    # Get the number of time steps in the data
    time_len = data.shape[0]
    # Calculate the size of the input window based on the stride size
    input_size = window_size-stride_size
    # Compute the number of windows for each time series
    windows_per_series = np.full((num_series), (time_len-input_size-target_window_size) // stride_size)
    # Adjust window count for training data
    if train: windows_per_series -= (data_start+stride_size-1) // stride_size
    # Calculate the total number of windows across all series
    total_windows = np.sum(windows_per_series)
    # Initialize the input matrix for the model
    x_input = np.zeros((total_windows, window_size, 1 + num_covariates), dtype='float32')
    # Initialize the labels matrix for the model
    label = np.zeros((total_windows, target_window_size, 1 + num_covariates), dtype='float32')
    # Initialize the matrix for additional input features
    v_input = np.zeros((total_windows, 2), dtype='float32')
    # Initialize counter for the number of processed windows
    count = 0
    # Iterate over each series
    for series in trange(num_series):
        # Iterate over each window in the current series
        for i in range(windows_per_series[series]):
            # Calculate the start of the current window
            window_start = stride_size*i+data_start[series] if train else stride_size*i
            # Calculate the end of the current window
            window_end = window_start+window_size
            # Calculate the end of the target window
            target_window_end = window_end+target_window_size
            # Set the current window data in the input matrix
            x_input[count, :, 0] = data[window_start:window_end, series]
            # Set the covariates for the current window in the input matrix
            x_input[count, :, 1:1+num_covariates] = covariates[window_start:window_end, :]
            # Set the target window data in the label matrix
            label[count, :, 0] = data[window_end:target_window_end, series]
            # Set the covariates for the target window in the label matrix
            label[count,:, 1:1+num_covariates] = covariates[window_end:target_window_end, :]
            # Calculate the sum of non-zero values in the input window
            nonzero_sum = (x_input[count, 1:input_size, 0]!=0).sum()
            # Normalize the input data if there are non-zero values
            if nonzero_sum == 0:
                v_input[count, 0] = 0
            else:
                v_input[count, 0] = np.true_divide(x_input[count, :input_size, 0].sum(),nonzero_sum)+1
                x_input[count, :, 0] = x_input[count, :, 0]/v_input[count, 0]
                label[count, :, 0] = label[count, :, 0]/v_input[count, 0]
            # Increment the count after processing a window
            count += 1
    # Return the prepared data matrices
    return x_input, v_input, label

In [2]:
# Generate covariates based on time attributes
def gen_covariates(times, num_covariates):
    # Initialize an array for storing covariates
    covariates = np.zeros((times.shape[0], num_covariates))
    # Iterate over each time point
    for i, input_time in enumerate(times):
        # Extract weekday, hour, and month as covariates
        covariates[i, 0] = input_time.weekday()
        covariates[i, 1] = input_time.hour
        covariates[i, 2] = input_time.month
    # Normalize the covariates using z-score normalization
    for i in range(num_covariates):
        covariates[:,i] = stats.zscore(covariates[:,i])
    # Return the covariates array
    return covariates[:, :num_covariates]


In [3]:
name = 'LD2011_2014.txt'
save_name = 'elect'
window_size = 192
stride_size = 24
target_window_size = 24
num_covariates = 3
train_start = '2011-01-01 00:00:00'
train_end = '2013-12-31 23:00:00'
validation_start = '2014-01-01 23:00:00'
validation_end = '2014-08-31 23:00:00'
test_start = '2014-08-25 00:00:00'
test_end = '2014-09-07 23:00:00'

save_path = os.path.join('data', save_name)
if not os.path.exists(save_path):
    os.makedirs(save_path)
csv_path = os.path.join(save_path, name)
if not os.path.exists(csv_path):
    zipurl = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00321/LD2011_2014.txt.zip'
    with urlopen(zipurl) as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            zfile.extractall(save_path)

data_frame = pd.read_csv(csv_path, sep=";", index_col=0, parse_dates=True, decimal=',')
data_frame = data_frame.resample('1H',label = 'left',closed = 'right').sum()[train_start:test_end]
data_frame.fillna(0, inplace=True)

covariates = gen_covariates(data_frame[train_start:test_end].index, num_covariates)

train_data = data_frame[train_start:train_end].values
validation_data = data_frame[validation_start:validation_end].values
test_data = data_frame[test_start:test_end].values

data_start = (train_data!=0).argmax(axis=0) #find first nonzero value in each time series
total_time = data_frame.shape[0] #32304
num_series = data_frame.shape[1] #370

X_train, v_train, y_train = prep_data(train_data, covariates, data_start)
X_validation, v_validation, y_validation = prep_data(validation_data, covariates, data_start, train=False)
X_test, v_test, y_test = prep_data(test_data, covariates, data_start, train=False)

100%|██████████| 370/370 [00:00<00:00, 4296.69it/s]


### Class VariationalDropout


In [7]:
# Variational Dropout implementation based on the paper: https://arxiv.org/abs/1512.05287
class VariationalDropout(nn.Module):
    # Initialize VariationalDropout with a specified dropout rate
    def __init__(self, dropout):
        super().__init__()
        self.dropout = dropout

    # Forward pass for applying dropout
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Return input as is if not in training mode
        if not self.training:
            return x
        # Determine the maximum batch size from input
        max_batch_size = x.size(1)
        # Generate a dropout mask with the same mask applied across the entire sequence
        m = x.new_empty(1, max_batch_size, x.size(2), requires_grad=False).bernoulli_(1 - self.dropout)
        # Apply the mask and scale the activations accordingly
        x = x.masked_fill(m == 0, 0) / (1 - self.dropout)
        return x


### Class LSTM


In [9]:
# Custom LSTM class with variational dropout for input, weights, and output
class LSTM(nn.LSTM):
    # Initialize LSTM with specific dropout rates for input, weight, and output
    def __init__(self, *args, dropouti=0., dropoutw=0., dropouto=0., unit_forget_bias=True, **kwargs):
        super().__init__(*args, **kwargs)
        self.unit_forget_bias = unit_forget_bias
        self.dropoutw = dropoutw
        self.input_drop = VariationalDropout(dropouti)
        self.output_drop = VariationalDropout(dropouto)

    # Function to apply dropout to hidden-to-hidden weights
    def _drop_weights(self):
        for name, param in self.named_parameters():
            if "weight_hh" in name:
                getattr(self, name).data = torch.nn.functional.dropout(param.data, p=self.dropoutw, training=self.training).contiguous()

    # Forward pass with custom dropout for LSTM
    def forward(self, input):
        # Apply dropout to weights
        self._drop_weights()
        # Apply variational dropout to input
        input = self.input_drop(input)
        # Standard LSTM forward pass
        seq, state = super().forward(input)
        # Apply variational dropout to output
        return self.output_drop(seq), state


 ### Class VDEncoder


In [10]:
# Variational Dropout Encoder module
class VDEncoder(nn.Module):
    # Initialize VDEncoder with specific input and output features and dropout probability
    def __init__(self, in_features, out_features, p):
        super(VDEncoder, self).__init__()
        # Define LSTM layers with variational dropout for the encoder
        self.model = nn.ModuleDict({
            'lstm1': LSTM(in_features, 32, dropouto=p),
            'lstm2': LSTM(32, 8, dropouto=p),
            'lstm3': LSTM(8, out_features, dropouto=p)
        })

    # Forward pass through the VDEncoder
    def forward(self, x):
        # Sequentially pass input through each LSTM layer
        out, _ = self.model['lstm1'](x)
        out, _ = self.model['lstm2'](out)
        out, _ = self.model['lstm3'](out)
        return out


### Class VDDecoder


In [11]:
# Variational Dropout Decoder module
class VDDecoder(nn.Module):
    # Initialize VDDecoder with dropout probability
    def __init__(self, p):
        super(VDDecoder, self).__init__()
        # Define LSTM layers with variational dropout for the decoder
        self.model = nn.ModuleDict({
            'lstm1': LSTM(1, 2, dropouto=p),
            'lstm2': LSTM(2, 2, dropouto=p),
            'lstm3': LSTM(2, 1, dropouto=p)
        })

    # Forward pass through the VDDecoder
    def forward(self, x):
        # Sequentially pass input through each LSTM layer
        out, _ = self.model['lstm1'](x)
        out, _ = self.model['lstm2'](out)
        out, _ = self.model['lstm3'](out)
        return out


### Class VDEncoderDecoder


In [12]:
# Define the Variational Dropout Encoder-Decoder model class.
class VDEncoderDecoder(nn.Module):
    # Initialize the class with specific input features, output steps, and dropout probability.
    def __init__(self, in_features, output_steps, p):
        super(VDEncoderDecoder, self).__init__()
        self.enc_in_features = in_features  # Store the number of input features for the encoder.
        self.output_steps = output_steps  # Store the number of output steps (f in the paper).
        self.enc_out_features = 1  # Define the number of output features for the encoder.
        self.traffic_col = 4  # Set the index of the traffic column (specific to use case).

        # Create a module dictionary to store encoder, decoder, and fully connected layers.
        self.model = nn.ModuleDict({
            'encoder': VDEncoder(self.enc_in_features, self.enc_out_features, p),  # Initialize the encoder.
            'decoder': VDDecoder(p),  # Initialize the decoder.
            'fc1': nn.Linear(window_size, 32),  # First fully connected layer to transform encoded output.
            'fc2': nn.Linear(32, self.output_steps)  # Second fully connected layer to generate final output.
        })

    # Define the forward pass of the model.
    def forward(self, x):
        # Pass the input through the encoder.
        out = self.model['encoder'](x)
        # Pass the encoder's output through the decoder.
        out = self.model['decoder'](out)
        # Flatten and pass the decoder's output through the first fully connected layer.
        out = self.model['fc1'](out.squeeze().view(64,-1))
        # Pass the output of the first fully connected layer through the second to get final predictions.
        out = self.model['fc2'](out)

        return out  # Return the final output.


### Class TrainDataset


In [14]:
# Define a dataset class for training.
class TrainDataset(Dataset):
    # Initialize the dataset with data and labels.
    def __init__(self, data, label):
        self.data = data  # Store the input data.
        self.label = label  # Store the labels corresponding to the data.
        self.train_len = self.data.shape[0]  # Calculate the length of the dataset.

    # Return the length of the dataset.
    def __len__(self):
        return self.train_len

    # Return the data and label for a given index.
    def __getitem__(self, index):
        # Return the main feature, additional covariates, and corresponding label.
        return (self.data[index,:,0],  # Extract the main feature from the data.
                self.data[index,:,1:1+num_covariates],  # Extract covariates from the data.
                self.label[index,:,0],  # Extract the main label.
                self.label[index,:,1:1+num_covariates])  # Extract covariates from the label.


### Class ValidationAndTestDataset

In [15]:
# Define a dataset class for validation and testing.
class ValidationAndTestDataset(Dataset):
    # Initialize the dataset with data, scaling factors (v), and labels.
    def __init__(self, data, v, label):
        self.data = data  # Store the input data.
        self.v = v  # Store the scaling factors for the data.
        self.label = label  # Store the labels corresponding to the data.
        self.test_len = self.data.shape[0]  # Calculate the length of the dataset.

    # Return the length of the dataset.
    def __len__(self):
        return self.test_len

    # Return the data, scaling factor, and label for a given index.
    def __getitem__(self, index):
        # Return the main feature, additional covariates, scaling factor, and corresponding label.
        return (self.data[index,:,0],  # Extract the main feature from the data.
                self.data[index,:,1:1+num_covariates],  # Extract covariates from the data.
                self.v[index],  # Extract the scaling factor for the given index.
                self.label[index,:,0],  # Extract the main label.
                self.label[index,:,1:1+num_covariates])  # Extract covariates from the label.


In [16]:
train_batch_size = 64

train_set = TrainDataset(data=X_train, label=y_train)
validation_set = ValidationAndTestDataset(data=X_validation, v=v_validation, label=y_validation)
test_set = ValidationAndTestDataset(data=X_test, v=v_test, label=y_test)

train_loader = DataLoader(train_set, batch_size=train_batch_size, drop_last=True)
validation_loader = DataLoader(validation_set, batch_size=train_batch_size, sampler=RandomSampler(test_set))
test_loader = DataLoader(test_set, batch_size=train_batch_size, sampler=RandomSampler(test_set), drop_last=True)

In [18]:
# Define a training function for the encoder-decoder model.
def train_encdec(model, device=torch.device('cuda'), num_epochs=2, learning_rate=1e-3):
    # Get the length of the training dataset.
    train_len = len(train_loader)
    # Initialize the optimizer with the model's parameters and the learning rate.
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    # Initialize a list to store loss values for each epoch.
    loss_summary = []
    # Set the loss function to mean squared error.
    loss_fn = F.mse_loss

    # Loop through the specified number of epochs.
    for epoch in range(num_epochs):
        # Set the model to training mode.
        model.train()
        # Initialize variables to track the sum of loss and total number of samples.
        epoch_loss_sum = 0.0
        total_sample = 0

        # Initialize a progress bar for the training loader.
        pbar = tqdm(train_loader)
        # Loop through each batch in the training dataset.
        for (train_batch, current_covs_batch, labels_batch, next_covs_batch) in pbar:
            # Determine batch size, sequence length, and horizon size.
            batch_size, seq_len, horizon_size = train_batch.shape[0], train_batch.shape[1], labels_batch.shape[0]
            # Update the total number of samples processed.
            total_sample += batch_size * seq_len * horizon_size
            # Reset the gradients of the model's parameters.
            optimizer.zero_grad()

            # Adjust the dimensions of the training batch and move it to the specified device.
            train_batch = train_batch.unsqueeze(2).permute(1,0,2).to(torch.float32).to(device)

            # Pass the training batch through the model to get the output.
            out = model(train_batch)
            # Calculate the loss between the model's output and the actual labels.
            loss = loss_fn(out.float(), labels_batch.squeeze().to(device).float())

            # Update the progress bar with the current loss value.
            pbar.set_description(f"Loss:{loss.item()}")
            # Perform backpropagation to calculate gradients.
            loss.backward()
            # Update the model's parameters based on the gradients.
            optimizer.step()

        # Store the loss of the current epoch.
        loss_summary.append(loss.cpu().detach())
    # Return the summary of losses after all epochs.
    return loss_summary


In [21]:
encdec_model = VDEncoderDecoder(in_features=1, output_steps=target_window_size, p=0.25).cuda()
train_encdec(encdec_model, num_epochs = 2, device=torch.device('cuda'))

Loss:0.08595262467861176: 100%|██████████| 5004/5004 [01:18<00:00, 63.81it/s]


[tensor(0.0760), tensor(0.0860)]

### class PredictionNetwork

In [22]:
# Define a prediction network class that inherits from nn.Module.
class PredictionNetwork(nn.Module):
    # Initialize the network with an encoder-decoder model and a dropout probability.
    def __init__(self, encoder_decoder, p=0.25):
        # Initialize the base class.
        super(PredictionNetwork, self).__init__()
        # Retrieve the encoder part of the passed encoder-decoder model and set it to evaluation mode.
        self.encoder = encoder_decoder.model['encoder'].eval()
        # Define a sequential model with linear layers, dropout, and ReLU activation functions.
        self.model = nn.Sequential(
            # First linear layer takes input size equal to window size times the number of features (1 + num_covariates).
            nn.Linear((1 + num_covariates)*window_size, 128),
            # Apply dropout with the specified probability.
            nn.Dropout(p),
            # Apply ReLU activation function.
            nn.ReLU(),
            # Apply dropout again.
            nn.Dropout(p),
            # Second linear layer to reduce dimension from 128 to 64.
            nn.Linear(128, 64),
            # Apply ReLU activation function.
            nn.ReLU(),
            # Apply dropout once more.
            nn.Dropout(p),
            # Final linear layer to produce output of size equal to target window size.
            nn.Linear(64, target_window_size)
        )

    # Define the forward pass of the model.
    def forward(self, x_input, cov_input):
        # Pass the input through the encoder to get extracted features.
        extracted = self.encoder(x_input)
        # Concatenate the extracted features with the covariate input.
        x_concat = torch.cat([extracted, cov_input], dim=-1)
        # Flatten the concatenated features and pass them through the sequential model.
        # Squeeze the output to remove dimensions of size 1.
        out = self.model(x_concat.view(train_batch_size, -1)).squeeze()
        # Return the final output.
        return out


In [23]:
# Define a function to train the prediction network.
def train_prediction_network(model, device=torch.device('cuda'), num_epochs = 2, learning_rate = 1e-3):
    # Calculate the total number of batches in the train loader.
    train_len = len(train_loader)
    # Set up the optimizer for the model.
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    # Initialize a list to keep track of the loss after each epoch.
    loss_summary = []
    # Define the loss function as mean squared error (MSE).
    loss_fn = F.mse_loss

    # Iterate over each epoch.
    for epoch in range(num_epochs):
        # Set the model to training mode.
        model.train()
        # Initialize variables to track loss and sample count.
        epoch_loss_sum = 0.0
        total_sample = 0

        # Create a progress bar for iterating through the data loader.
        pbar = tqdm(train_loader)
        # Iterate over each batch in the data loader.
        for (train_batch, current_covs_batch, labels_batch, next_covs_batch) in pbar:
            # Determine the size of the batch, sequence length, and horizon size.
            batch_size, seq_len, horizon_size = train_batch.shape[0], train_batch.shape[1], labels_batch.shape[0]
            # Calculate the total number of samples processed.
            total_sample += batch_size * seq_len * horizon_size
            # Reset gradients in the optimizer.
            optimizer.zero_grad()

            # Prepare the training batch and covariates batch for the model, adjusting dimensions and moving to the device.
            train_batch = train_batch.unsqueeze(2).permute(1,0,2).to(torch.float32).to(device)
            current_covs_batch = current_covs_batch.permute(1,0,2).to(torch.float32).to(device)

            # Pass the data through the model.
            out = model(train_batch, current_covs_batch)
            # Calculate the loss between model output and labels.
            loss = loss_fn(out.float(), labels_batch.squeeze().to(device).float())

            # Update the progress bar with the current loss.
            pbar.set_description(f"Loss:{loss.item()}")
            # Backpropagate the loss.
            loss.backward()
            # Update model parameters.
            optimizer.step()

        # Store the loss of the last batch in the summary.
        loss_summary.append(loss.cpu().detach())
    # Return the loss summary and the optimizer for further use.
    return loss_summary, optimizer

# Define a function to evaluate the prediction network.
def evaluate_prediction_network(model, optimizer, device=torch.device('cuda')):
    # Set the criterion as mean squared error (MSE) loss for evaluation.
    criterion = nn.MSELoss()
    # Initialize a list to store root mean squared error (RMSE) results for each batch.
    rmse_results = []

    # Disable gradient calculations for evaluation.
    with torch.no_grad():
        # Set the model to evaluation mode.
        model.eval()
        # Initialize an array to store loss per epoch.
        loss_epoch = np.zeros(len(train_loader))

        # Create a progress bar for iterating through the test data loader.
        pbar = tqdm(test_loader)
        # Iterate over each batch in the test loader.
        for (ts_data_batch, current_covs_batch, v_batch, labels_batch, next_covs_batch) in pbar:
            # Reset gradients in the optimizer.
            optimizer.zero_grad()
            # Prepare the test data batch and covariates batch for the model, adjusting dimensions and moving to the device.
            ts_data_batch = ts_data_batch.unsqueeze(2).permute(1,0,2).to(torch.float32).to(device)
            current_covs_batch = current_covs_batch.permute(1,0,2).to(torch.float32).to(device)

            # Pass the data through the model.
            out = model(ts_data_batch, current_covs_batch)
            # Calculate RMSE for the batch and add it to the results.
            rmse_results.append(torch.sqrt(criterion(out.detach().cpu(), labels_batch)).item())

    # Calculate the average RMSE across all test batches.
    test_rmse = np.mean(rmse_results)
    # Return the average test RMSE.
    return test_rmse


In [24]:
# Define a function to train the prediction network.
def train_prediction_network(model, device=torch.device('cuda'), num_epochs = 2, learning_rate = 1e-3):
    # Calculate the total number of batches in the train loader.
    train_len = len(train_loader)
    # Set up the optimizer for the model.
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    # Initialize a list to keep track of the loss after each epoch.
    loss_summary = []
    # Define the loss function as mean squared error (MSE).
    loss_fn = F.mse_loss

    # Iterate over each epoch.
    for epoch in range(num_epochs):
        # Set the model to training mode.
        model.train()
        # Initialize variables to track loss and sample count.
        epoch_loss_sum = 0.0
        total_sample = 0

        # Create a progress bar for iterating through the data loader.
        pbar = tqdm(train_loader)
        # Iterate over each batch in the data loader.
        for (train_batch, current_covs_batch, labels_batch, next_covs_batch) in pbar:
            # Determine the size of the batch, sequence length, and horizon size.
            batch_size, seq_len, horizon_size = train_batch.shape[0], train_batch.shape[1], labels_batch.shape[0]
            # Calculate the total number of samples processed.
            total_sample += batch_size * seq_len * horizon_size
            # Reset gradients in the optimizer.
            optimizer.zero_grad()

            # Prepare the training batch and covariates batch for the model, adjusting dimensions and moving to the device.
            train_batch = train_batch.unsqueeze(2).permute(1,0,2).to(torch.float32).to(device)
            current_covs_batch = current_covs_batch.permute(1,0,2).to(torch.float32).to(device)

            # Pass the data through the model.
            out = model(train_batch, current_covs_batch)
            # Calculate the loss between model output and labels.
            loss = loss_fn(out.float(), labels_batch.squeeze().to(device).float())

            # Update the progress bar with the current loss.
            pbar.set_description(f"Loss:{loss.item()}")
            # Backpropagate the loss.
            loss.backward()
            # Update model parameters.
            optimizer.step()

        # Store the loss of the last batch in the summary.
        loss_summary.append(loss.cpu().detach())
    # Return the loss summary and the optimizer for further use.
    return loss_summary, optimizer

# Define a function to evaluate the prediction network.
def evaluate_prediction_network(model, optimizer, device=torch.device('cuda')):
    # Set the criterion as mean squared error (MSE) loss for evaluation.
    criterion = nn.MSELoss()
    # Initialize a list to store root mean squared error (RMSE) results for each batch.
    rmse_results = []

    # Disable gradient calculations for evaluation.
    with torch.no_grad():
        # Set the model to evaluation mode.
        model.eval()
        # Initialize an array to store loss per epoch.
        loss_epoch = np.zeros(len(train_loader))

        # Create a progress bar for iterating through the test data loader.
        pbar = tqdm(test_loader)
        # Iterate over each batch in the test loader.
        for (ts_data_batch, current_covs_batch, v_batch, labels_batch, next_covs_batch) in pbar:
            # Reset gradients in the optimizer.
            optimizer.zero_grad()
            # Prepare the test data batch and covariates batch for the model, adjusting dimensions and moving to the device.
            ts_data_batch = ts_data_batch.unsqueeze(2).permute(1,0,2).to(torch.float32).to(device)
            current_covs_batch = current_covs_batch.permute(1,0,2).to(torch.float32).to(device)

            # Pass the data through the model.
            out = model(ts_data_batch, current_covs_batch)
            # Calculate RMSE for the batch and add it to the results.
            rmse_results.append(torch.sqrt(criterion(out.detach().cpu(), labels_batch)).item())

    # Calculate the average RMSE across all test batches.
    test_rmse = np.mean(rmse_results)
    # Return the average test RMSE.
    return test_rmse


In [ ]:
def accuracy_RMSE(mu: torch.Tensor, labels: torch.Tensor, relative = False):
    zero_index = (labels != 0)
    diff = torch.sum(torch.mul((mu[zero_index] - labels[zero_index]), (mu[zero_index] - labels[zero_index]))).item()
    if relative is False:
        return [diff, torch.sum(zero_index).item(), torch.sum(zero_index).item()]
    else:
        summation = torch.sum(torch.abs(labels[zero_index])).item()
        if summation == 0:
            logger.error('summation denominator error! ')
        return [diff, summation, torch.sum(zero_index).item()]

def update_metrics(raw_metrics, sample_mu, labels, relative=False):
    # TODO: use samples to calcualte rou50, rou90 metrics
    raw_metrics['RMSE'] = raw_metrics['RMSE'] + accuracy_RMSE(sample_mu, labels, relative=relative)
    return raw_metrics

def final_metrics(raw_metrics):
    summary_metric = {}
    summary_metric['RMSE'] = np.sqrt(raw_metrics['RMSE'][0] / raw_metrics['RMSE'][2]) / (
                raw_metrics['RMSE'][1] / raw_metrics['RMSE'][2])
    return summary_metric

def dropout_on(m):
    if type(m) in [torch.nn.Dropout, LSTM]:
        m.train()

def dropout_off(m):
    if type(m) in [torch.nn.Dropout, LSTM]:
        m.eval()

def mc_dropout(model, B, device):
    model = model.apply(dropout_on)

    pbar = range(B)
    pbar = tqdm(pbar)

    y_hats = []
    for b in pbar:
        for (x, cov, v, y, ncov) in test_loader:
            x,cov,y = x.to(device), cov.to(device), y.to(device)
            break
        x = x.unsqueeze(2).permute(1, 0, 2).to(torch.float32).to(device)
        cov = cov.permute(1,0,2).to(torch.float32).to(device)

        y_hat_b = model(x, cov).float()
        y_hats.append(y_hat_b.cpu().detach().numpy())

    ymc_hats = np.mean(y_hats, axis=0)
    eta_1s   = np.mean((ymc_hats[:,0] - np.stack(y_hats)[:,:,0])**2, axis=0)
    return ymc_hats, eta_1s


def inference(model, B=100, device=torch.device('cuda')):
    # mc dropout
    ymc_hats, eta_1s = mc_dropout(model, B, device)

    # inherent noise
    model.apply(dropout_off)
    for (x, cov, v, y, ncov) in validation_loader:
        x,cov,y = x.to(device), cov.to(device), y.to(device)
        break
    x = x.unsqueeze(2).permute(1, 0, 2).to(torch.float32).to(device)
    cov = cov.permute(1,0,2).to(torch.float32).to(device)
    y_hat_b = model(x, cov)

    eta_2sq = np.mean(y_hat_b.cpu().detach().numpy()[:,0])
    # total noise
    etas = np.sqrt(eta_1s + eta_2sq)

    for (x, cov, v, y, ncov) in test_loader:
        break
    normalized_label = y.T/v[:,0]
    diff = torch.nansum(torch.mul((normalized_label.T - torch.tensor(ymc_hats)), (normalized_label.T - torch.tensor(ymc_hats)))).item()
    test_rmse = np.sqrt(diff/len(test_set))

    return test_rmse

In [25]:
# Define a function to calculate the root mean squared error (RMSE).
def accuracy_RMSE(mu: torch.Tensor, labels: torch.Tensor, relative = False):
    # Identify non-zero elements in labels.
    zero_index = (labels != 0)
    # Calculate the squared difference between predictions and labels for non-zero elements.
    diff = torch.sum(torch.mul((mu[zero_index] - labels[zero_index]), (mu[zero_index] - labels[zero_index]))).item()
    # Return different metrics based on whether relative error is required.
    if relative is False:
        return [diff, torch.sum(zero_index).item(), torch.sum(zero_index).item()]
    else:
        # Calculate the sum of absolute values of labels for non-zero elements.
        summation = torch.sum(torch.abs(labels[zero_index])).item()
        # Log an error if the summation is zero.
        if summation == 0:
            logger.error('summation denominator error! ')
        return [diff, summation, torch.sum(zero_index).item()]

# Define a function to update the raw metrics with new data.
def update_metrics(raw_metrics, sample_mu, labels, relative=False):
    # Update the RMSE metric in the raw metrics dictionary.
    raw_metrics['RMSE'] = raw_metrics['RMSE'] + accuracy_RMSE(sample_mu, labels, relative=relative)
    return raw_metrics

# Define a function to calculate final metrics from raw metrics.
def final_metrics(raw_metrics):
    summary_metric = {}
    # Calculate the final RMSE value.
    summary_metric['RMSE'] = np.sqrt(raw_metrics['RMSE'][0] / raw_metrics['RMSE'][2]) / (
                raw_metrics['RMSE'][1] / raw_metrics['RMSE'][2])
    return summary_metric

# Define a function to enable dropout in a model.
def dropout_on(m):
    if type(m) in [torch.nn.Dropout, LSTM]:
        m.train()

# Define a function to disable dropout in a model.
def dropout_off(m):
    if type(m) in [torch.nn.Dropout, LSTM]:
        m.eval()

# Define a function to perform Monte Carlo dropout.
def mc_dropout(model, B, device):
    # Enable dropout in the model.
    model = model.apply(dropout_on)

    # Initialize a progress bar for Monte Carlo iterations.
    pbar = range(B)
    pbar = tqdm(pbar)

    y_hats = []
    # Iterate over each Monte Carlo sample.
    for b in pbar:
        # Retrieve a batch of data from the test loader.
        for (x, cov, v, y, ncov) in test_loader:
            x,cov,y = x.to(device), cov.to(device), y.to(device)
            break
        # Prepare the data for the model.
        x = x.unsqueeze(2).permute(1, 0, 2).to(torch.float32).to(device)
        cov = cov.permute(1,0,2).to(torch.float32).to(device)

        # Make predictions with the model.
        y_hat_b = model(x, cov).float()
        # Store the predictions.
        y_hats.append(y_hat_b.cpu().detach().numpy())

    # Calculate the mean prediction over all Monte Carlo samples.
    ymc_hats = np.mean(y_hats, axis=0)
    # Calculate the variance of the first component across samples.
    eta_1s   = np.mean((ymc_hats[:,0] - np.stack(y_hats)[:,:,0])**2, axis=0)
    return ymc_hats, eta_1s


def inference(model, B=100, device=torch.device('cuda')):
    # mc dropout
    ymc_hats, eta_1s = mc_dropout(model, B, device)

    # inherent noise
    model.apply(dropout_off)
    for (x, cov, v, y, ncov) in validation_loader:
        x,cov,y = x.to(device), cov.to(device), y.to(device)
        break
    x = x.unsqueeze(2).permute(1, 0, 2).to(torch.float32).to(device)
    cov = cov.permute(1,0,2).to(torch.float32).to(device)
    y_hat_b = model(x, cov)

    eta_2sq = np.mean(y_hat_b.cpu().detach().numpy()[:,0])
    # total noise
    etas = np.sqrt(eta_1s + eta_2sq)

    for (x, cov, v, y, ncov) in test_loader:
        break
    normalized_label = y.T/v[:,0]
    diff = torch.nansum(torch.mul((normalized_label.T - torch.tensor(ymc_hats)), (normalized_label.T - torch.tensor(ymc_hats)))).item()
    test_rmse = np.sqrt(diff/len(test_set))

    return test_rmse

In [26]:
prednet_model = PredictionNetwork(encdec_model).cuda()
loss_summary, optimizer = train_prediction_network(prednet_model, num_epochs = 2, device=torch.device('cuda'))
evaluate_prediction_network(prednet_model, optimizer)
test_rmse = inference(prednet_model, B=500, device=torch.device('cuda'))

100%|██████████| 500/500 [00:02<00:00, 223.80it/s]


In [27]:
print(test_rmse)

0.8222752754229762
